In [96]:
import sys
import os
import dbfread
from dbfread import DBF
import dbf
from glob import glob 
import pandas as pd
import csv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from ipywidgets import interact
import seaborn as sns
from datetime import datetime, timedelta
import time
import pickle
from collections import Counter
from tqdm import tqdm
import plotly.express as px
import datetime
from datetime import datetime
import matplotlib.font_manager as fm
# font_list = fm.findSystemFonts(fontpaths=None, fontext = 'ttf')
from matplotlib import font_manager
# font_path = "C:/Windows/Fonts/NGULIM.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)
rc('font', family='AppleGothic')
import heartpy as hp
import sklearn
import sklearn.preprocessing
from sklearn.preprocessing import normalize
import pywt

In [97]:
import pandas as pd

from scipy.interpolate import CubicSpline
from scipy.signal import medfilt

from scipy.interpolate import UnivariateSpline
from scipy.signal import welch, periodogram

import heartpy as hp

import warnings

warnings.filterwarnings('ignore', category=UserWarning)

## Data Loading

In [98]:
WEAROS_PPG_DATA = pd.read_csv('../WearOS_Dataset/PPG_Dataset.csv')
WEAROS_PPG_DATA.head()

,timestamp,ppg,pnum,condition
0,1.671078e+09,3.017970e-39,1,baseline1
1,1.671078e+09,3.017630e-39,1,baseline1
2,1.671078e+09,3.018150e-39,1,baseline1
3,1.671078e+09,3.018690e-39,1,baseline1
4,1.671078e+09,3.019180e-39,1,baseline1


In [99]:
CONDITION_TIMESTAMP = pd.read_csv('../WearOS_Dataset/condition_timestamp.csv')
CONDITION_TIMESTAMP.head()

,condition,timestamp,pnum
0,b1_start,1.671078e+09,1
1,b1_end,1.671078e+09,1
2,b2_start,1.671078e+09,1
3,b2_end,1.671078e+09,1
4,s_start,1.671079e+09,1


In [100]:
pnum_list = range(1,13,1)
cond_list = ['baseline1', 'baseline2', 'stress']

# Signal Preprocessing & Peak Detection & Artifact Removal

In [223]:
# WEAROS_PPG_DATA['filtered_ppg'] = hp.filter_signal(np.array(WEAROS_PPG_DATA.loc[:,'ppg']), cutoff=[0.8,3.5], sample_rate=25, filtertype='bandpass')


PP_INTERVAL_PROCESSED = pd.DataFrame()

for pnum in pnum_list:
    start_timestamp = CONDITION_TIMESTAMP.loc[(CONDITION_TIMESTAMP['pnum']==pnum) & (CONDITION_TIMESTAMP['condition'].str.contains('start')), 'timestamp']
    end_timestamp = CONDITION_TIMESTAMP.loc[(CONDITION_TIMESTAMP['pnum']==pnum) & (CONDITION_TIMESTAMP['condition'].str.contains('end')), 'timestamp']

    # Cut data by pnum
    WEAROS_PPG_DATA_p = WEAROS_PPG_DATA.loc[WEAROS_PPG_DATA['pnum'] == pnum]
    WEAROS_PPG_DATA_p = WEAROS_PPG_DATA_p.reset_index(drop=True)

    # Filtering
    WEAROS_PPG_DATA_p['filtered_ppg'] = hp.filter_signal(np.array(WEAROS_PPG_DATA_p.loc[:,'ppg']), order=3, cutoff=[0.8,3.5], sample_rate=25, filtertype='bandpass')

    for cond, cond_start_timestamp, cond_end_timestamp in zip(cond_list, start_timestamp, end_timestamp):
        # Cut data by condition
        WEAROS_PPG_DATA_cond = WEAROS_PPG_DATA_p.loc[WEAROS_PPG_DATA_p['condition'] == cond]
        WEAROS_PPG_DATA_cond = WEAROS_PPG_DATA_cond.reset_index(drop=True)

        # Outlier Removal
        WEAROS_PPG_DATA_cond['filtered_and_outlier_removed_ppg'] = hp.datautils.outliers_modified_z(np.array(WEAROS_PPG_DATA_cond['filtered_ppg']))[0]     

        # Peak Detection
        PP_INTERVAL = pd.DataFrame()
        input = WEAROS_PPG_DATA_cond['filtered_and_outlier_removed_ppg']
        working_data, measures = hp.process(input, sample_rate=25)
        PP_INTERVAL['RR_list'] = working_data['RR_list']
        timestamp= []
        for peak in working_data['peaklist'][:-1]:
            timestamp.append(WEAROS_PPG_DATA_cond.iloc[peak, 0])
        PP_INTERVAL['timestamp'] = timestamp

        # Chunking-based Correction
        PP_INTERVAL = PP_INTERVAL.loc[(PP_INTERVAL['RR_list'] > 300) & (PP_INTERVAL['RR_list'] < 1500)]
        PP_INTERVAL_PROCESSED_1 = pd.DataFrame()
        chunk_start = cond_start_timestamp
        while True:
            chunk_mid = chunk_start + 4
            chunk_end = chunk_start + 8
            chunk_1 = PP_INTERVAL.loc[(PP_INTERVAL['timestamp'] > chunk_start) & (PP_INTERVAL['timestamp'] < chunk_mid)]
            mean_rr = chunk_1['RR_list'].mean()
            chunk_2 = PP_INTERVAL.loc[(PP_INTERVAL['timestamp'] > chunk_mid) & (PP_INTERVAL['timestamp'] < chunk_end)]
            chunk_2 = chunk_2.loc[(chunk_2['RR_list'] > (mean_rr - 150)) & (chunk_2['RR_list'] < (mean_rr + 150))]
            PP_INTERVAL_PROCESSED_1 = pd.concat([PP_INTERVAL_PROCESSED_1, chunk_2], ignore_index=True)
            chunk_start = chunk_mid
            if (chunk_start + 4) >= cond_end_timestamp:
                break
        PP_INTERVAL_PROCESSED_2 = pd.DataFrame(columns=['RR_list','timestamp'])
        for idx in range(0,len(PP_INTERVAL_PROCESSED_1)-2):
            start = PP_INTERVAL_PROCESSED_1.iloc[idx]
            mid = PP_INTERVAL_PROCESSED_1.iloc[idx+1]
            PP_INTERVAL_PROCESSED_2.loc[len(PP_INTERVAL_PROCESSED_2)] = mid
            end = PP_INTERVAL_PROCESSED_1.iloc[idx+2]
            if (end['timestamp']-mid['timestamp']) > 2:
                gap = (end['timestamp']-mid['timestamp'])*1000
                peak_num = int((2*gap) // (end['RR_list']+start['RR_list']))
                rr = gap // (peak_num+1)
                for i in range(peak_num):
                    row = {'RR_list': rr, 'timestamp': mid['timestamp']+(rr*(i+1))/1000}
                    PP_INTERVAL_PROCESSED_2.loc[len(PP_INTERVAL_PROCESSED_2)] = row

        # Filtering-based Correction
        if (len(PP_INTERVAL_PROCESSED_2) > 2):
            med_filt = medfilt(PP_INTERVAL_PROCESSED_2['RR_list'], kernel_size=7)
            PP_INTERVAL_PROCESSED_2['med_filt'] = med_filt
            wrong = PP_INTERVAL_PROCESSED_2.loc[abs(PP_INTERVAL_PROCESSED_2['RR_list'] - PP_INTERVAL_PROCESSED_2['med_filt']) > 150]
            wrong_idx = PP_INTERVAL_PROCESSED_2.index.isin(list(wrong.index))
            cs = CubicSpline(PP_INTERVAL_PROCESSED_2[~wrong_idx]['timestamp'], PP_INTERVAL_PROCESSED_2[~wrong_idx]['RR_list'])
            new_rr = np.linspace(PP_INTERVAL_PROCESSED_2[~wrong_idx]['timestamp'].min(), PP_INTERVAL_PROCESSED_2[~wrong_idx]['timestamp'].max(), len(PP_INTERVAL_PROCESSED_2))
            PP_INTERVAL_PROCESSED_2['RR_list'] = cs(new_rr)

        # Galaxy Final Merge
        PP_INTERVAL_PROCESSED_2['pnum'] = pnum
        PP_INTERVAL_PROCESSED_2['cond'] = cond
        PP_INTERVAL_PROCESSED = pd.concat([PP_INTERVAL_PROCESSED, PP_INTERVAL_PROCESSED_2], ignore_index=True)

In [224]:
PP_INTERVAL_PROCESSED

,RR_list,timestamp,med_filt,pnum,cond
0,720.000000,1.671078e+09,720.0,1,baseline1
1,760.318172,1.671078e+09,760.0,1,baseline1
2,799.323354,1.671078e+09,760.0,1,baseline1
3,802.807198,1.671078e+09,760.0,1,baseline1
4,766.726167,1.671078e+09,800.0,1,baseline1
...,...,...,...,...,...
7717,601.857888,1.671673e+09,600.0,12,stress
7718,597.503495,1.671673e+09,600.0,12,stress
7719,638.731519,1.671673e+09,600.0,12,stress
7720,640.424137,1.671673e+09,600.0,12,stress


# Feature Extraction

In [225]:
def calc_ts_measures(rr_list, rr_diff, rr_sqdiff, measures={}, working_data={}):
    '''
    Function that calculates the time-series HRV measurements.
    '''
    measures['bpm'] = 60000 / np.mean(rr_list)
    measures['ibi'] = np.mean(rr_list)

    measures['sdnn'] = np.std(rr_list)
    measures['sdsd'] = np.std(rr_diff)
    measures['rmssd'] = np.sqrt(np.mean(rr_sqdiff))
    nn20 = rr_diff[np.where(rr_diff > 20.0)]
    nn50 = rr_diff[np.where(rr_diff > 50.0)]
    working_data['nn50'] = nn50
    try:
        measures['pnn50'] = float(len(nn50)) / float(len(rr_diff))
    except:
        measures['pnn50'] = np.nan

    return working_data, measures

def calc_fd_measures(rr_list, method='welch', welch_wsize=240, square_spectrum=False, measures={}, degree_smoothing_spline=3):
    '''
    Function that calculates the frequency-domain HRV measurements.
    '''
    if len(rr_list) <= 1:
        measures['lf'] = np.nan
        measures['hf'] = np.nan
        measures['lf_hf'] = np.nan
        return measures

    rr_x = np.cumsum(rr_list)

    resamp_factor = 4
    datalen = int((len(rr_x) - 1)*resamp_factor)
    rr_x_new = np.linspace(int(rr_x[0]), int(rr_x[-1]), datalen)

    if len(rr_x) > degree_smoothing_spline:
        interpolation_func = UnivariateSpline(rr_x, rr_list, k=degree_smoothing_spline)
        rr_interp = interpolation_func(rr_x_new)

        # RR-list in units of ms, with the sampling rate at 1 sample per beat
        dt = np.mean(rr_list) / 1000  # in sec
        fs = 1 / dt  # about 1.1 Hz; 50 BPM would be 0.83 Hz, just enough to get the max of the HF band at 0.4 Hz according to Nyquist
        fs_new = fs * resamp_factor

        # compute PSD (one-sided, units of ms^2/Hz)
        if method == 'fft':
            frq = np.fft.fftfreq(datalen, d=(1 / fs_new))
            frq = frq[range(int(datalen / 2))]
            Y = np.fft.fft(rr_interp) / datalen
            Y = Y[range(int(datalen / 2))]
            psd = np.power(Y, 2)

        elif method == 'periodogram':
            frq, psd = periodogram(rr_interp, fs=fs_new)

        elif method == 'welch':
            # nperseg should be based on trade-off btw temporal res and freq res
            nperseg = welch_wsize * fs_new
            if nperseg >= len(rr_x_new):  # if nperseg is larger than the available data segment
                nperseg = len(rr_x_new)  # set it to length of data segment to prevent scipy warnings
            frq, psd = welch(rr_interp, fs=fs_new, nperseg=nperseg)
        else:
            raise ValueError("specified method incorrect, use 'fft', 'periodogram' or 'welch'")

        df = frq[1] - frq[0]
        measures['lf'] = np.trapz(abs(psd[(frq >= 0.04) & (frq < 0.15)]), dx=df)
        measures['hf'] = np.trapz(abs(psd[(frq >= 0.15) & (frq < 0.4)]), dx=df)
        measures['lf_hf'] = measures['lf'] / measures['hf']

    else:
        measures['lf'] = np.nan
        measures['hf'] = np.nan
        measures['lf_hf'] = np.nan

    return measures

In [226]:
window_size = 60
stepsize = 0.25

HRV_FEATURES_DATAFRAME = pd.DataFrame(columns=['ibi','bpm','sdnn','rmssd','pnn50','lf','hf','lf_hf','pnum','cond'])

for pnum in pnum_list:
    for cond in cond_list:
        galaxy = PP_INTERVAL_PROCESSED.loc[(PP_INTERVAL_PROCESSED['pnum']==pnum)&(PP_INTERVAL_PROCESSED['cond']==cond)]
        window_start = galaxy['timestamp'].min()
        add_g = pd.DataFrame(columns=['ibi','bpm','sdnn','rmssd','pnn50','lf','hf','lf_hf'])
        while True:
            window_end = window_start + window_size
            window = galaxy.loc[(galaxy['timestamp'] > window_start) & (galaxy['timestamp'] < window_end)]
            rr_list = window['RR_list']
            rr_diff = np.abs(np.diff(rr_list))
            rr_sqdiff = np.power(rr_diff, 2)
            wd_ts, m_ts = calc_ts_measures(rr_list, rr_diff, rr_sqdiff)
            m = calc_fd_measures(rr_list=np.asarray(rr_list), measures=m_ts)
            add_g.loc[len(add_g)] = m
            window_start += stepsize
            if (window_start + 60) > galaxy['timestamp'].max():
                break
        add_g['pnum'] = pnum
        add_g['cond'] = cond
        HRV_FEATURES_DATAFRAME = pd.concat([HRV_FEATURES_DATAFRAME, add_g], ignore_index=True)

HRV_FEATURES_DATAFRAME.head()

,ibi,bpm,sdnn,rmssd,pnn50,lf,hf,lf_hf,pnum,cond
0,776.518957,77.267914,62.179649,50.593609,0.270270,1329.770144,2273.742878,0.584838,1,baseline1
1,776.498793,77.269920,61.769464,50.416174,0.266667,1347.546972,2312.610569,0.582695,1,baseline1
2,776.498793,77.269920,61.769464,50.416174,0.266667,1347.546972,2312.610569,0.582695,1,baseline1
3,776.714535,77.248458,62.151446,50.552741,0.270270,1389.742245,2220.806877,0.625783,1,baseline1
4,775.897449,77.329807,62.145378,50.696151,0.280000,1407.374422,2259.983299,0.622737,1,baseline1


# Stress Detection Model

## Data Normalization

In [227]:
from sklearn.preprocessing import StandardScaler

feature_list = ['ibi','bpm','sdnn','rmssd','pnn50','lf','hf','lf_hf']
HRV_FEATURES_DATAFRAME_NORMALIZED = pd.DataFrame(columns=feature_list)

for pnum in pnum_list:
    HRV_FEATURES_DATAFRAME_p = HRV_FEATURES_DATAFRAME.loc[HRV_FEATURES_DATAFRAME['pnum']==pnum]
    scaled = StandardScaler().fit_transform(HRV_FEATURES_DATAFRAME_p[feature_list])
    scaled_df = pd.DataFrame(scaled, index=HRV_FEATURES_DATAFRAME_p.index, columns=feature_list)
    HRV_FEATURES_DATAFRAME_NORMALIZED = pd.concat([HRV_FEATURES_DATAFRAME_NORMALIZED, scaled_df], ignore_index=True)

HRV_FEATURES_DATAFRAME_NORMALIZED = HRV_FEATURES_DATAFRAME_NORMALIZED.reset_index(drop=True)
HRV_FEATURES_DATAFRAME_NORMALIZED['cond'] = list(HRV_FEATURES_DATAFRAME['cond'])        
HRV_FEATURES_DATAFRAME_NORMALIZED['pnum'] = list(HRV_FEATURES_DATAFRAME['pnum'])
HRV_FEATURES_DATAFRAME_NORMALIZED['label'] = HRV_FEATURES_DATAFRAME['cond'].apply(lambda x: 1 if 'stress' in x else 0)

HRV_FEATURES_DATAFRAME_NORMALIZED.head()

,ibi,bpm,sdnn,rmssd,pnn50,lf,hf,lf_hf,cond,pnum,label
0,2.052504,-1.892195,-0.378220,0.182108,0.440094,0.457753,0.066449,-0.641665,baseline1,1,0
1,2.052089,-1.891873,-0.396956,0.161832,0.346316,0.480915,0.070308,-0.645909,baseline1,1,0
2,2.052089,-1.891873,-0.396956,0.161832,0.346316,0.480915,0.070308,-0.645909,baseline1,1,0
3,2.056536,-1.895312,-0.379508,0.177437,0.440094,0.535892,0.061193,-0.560552,baseline1,1,0
4,2.039691,-1.882278,-0.379785,0.193825,0.693297,0.558865,0.065083,-0.566586,baseline1,1,0


## Simple Threshold-based Model

In [253]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

def calculate_best_threshold(INPUT_DATAFRAME):
    best_threshold_dict = {}

    for idx, feature in enumerate(feature_list):
        min = INPUT_DATAFRAME[feature].min()
        max = INPUT_DATAFRAME[feature].max()

        fpr_list, tpr_list, gmeans_list = [], [], []
        threshold_list = np.linspace(min, max, 100)

        for threshold in threshold_list:
            MODEL_DEVICE = pd.DataFrame(columns=['label',feature])

            MODEL_DEVICE['label'] = INPUT_DATAFRAME['label']
            if feature in ['ibi','sdnn','rmssd','pnn50','hf']:
                MODEL_DEVICE[feature] = INPUT_DATAFRAME[feature].apply(lambda x: 0 if x > threshold else 1)
            else:
                MODEL_DEVICE[feature] = INPUT_DATAFRAME[feature].apply(lambda x: 0 if x < threshold else 1)

            cf = confusion_matrix(list(MODEL_DEVICE['label']), list(MODEL_DEVICE[feature]))
            fpr = cf[0][1] / (cf[0][1] + cf[0][0])
            tpr = cf[1][1] / (cf[1][1] + cf[1][0])

            fpr_list.append(fpr)
            tpr_list.append(tpr)
            gmeans_list.append(np.sqrt(tpr * (1-fpr)))

        ix = np.argmax(gmeans_list)
        best_threshold = threshold_list[ix]

        best_threshold_dict[feature] = best_threshold

    return best_threshold_dict

def threshold_model(INPUT_DATAFRAME):
    FEATURES = INPUT_DATAFRAME[feature_list]
    LABELS_AVAL = INPUT_DATAFRAME['label']
    PIDS = INPUT_DATAFRAME['pnum']

    RESULTS_AVAL, MODELS_AVAL, TRAIN_SETS_AVAL = [], {}, {}

    splitter = LeaveOneGroupOut()

    for I_train, I_test in splitter.split(FEATURES, LABELS_AVAL, PIDS):    
        X_train, X_test = FEATURES.iloc[I_train], FEATURES.iloc[I_test] 
        y_train, y_test = LABELS_AVAL.iloc[I_train], LABELS_AVAL.iloc[I_test]
        test_name = np.unique(PIDS[I_test])[0]

        input = pd.concat([X_train, y_train], axis=1)
        clf = calculate_best_threshold(input)

        for feature in feature_list:
            best_threshold = clf[feature]
            
            if feature in ['ibi', 'sdnn', 'rmssd', 'pnn50', 'hf']:
                y_pred = X_test[feature].apply(lambda x: 0 if x > best_threshold else 1)
            else:
                y_pred = X_test[feature].apply(lambda x: 0 if x < best_threshold else 1)

            RESULTS_AVAL.append({
                'feature': feature,
                'accuracy': accuracy_score(y_test, y_pred),
                'f1_score': f1_score(y_test, y_pred),
            })

            MODELS_AVAL[test_name] = clf
            TRAIN_SETS_AVAL[test_name] = X_train

    return RESULTS_AVAL

In [254]:
RESULTS_DATAFRAME = threshold_model(HRV_FEATURES_DATAFRAME_NORMALIZED)
print(pd.DataFrame(RESULTS_DATAFRAME).groupby(by='feature').mean())
print(pd.DataFrame(RESULTS_DATAFRAME).groupby(by='feature').std())

         accuracy  f1_score
feature                    
bpm      0.791737  0.806828
hf       0.389983  0.063780
ibi      0.796143  0.809909
lf       0.479136  0.309937
lf_hf    0.525358  0.580969
pnn50    0.428790  0.508402
rmssd    0.364935  0.414337
sdnn     0.411928  0.512379
         accuracy  f1_score
feature                    
bpm      0.234654  0.226506
hf       0.040477  0.150436
ibi      0.235497  0.230540
lf       0.132100  0.338438
lf_hf    0.108359  0.095245
pnn50    0.126482  0.091124
rmssd    0.173937  0.171082
sdnn     0.176699  0.141203


## Machine-Learning Model

In [250]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit, LeaveOneGroupOut
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score

def major_voting(INPUT_DATAFRAME):
    DATA_SET = INPUT_DATAFRAME
    RESULTS_AVAL = []

    for pnum in pnum_list:
        y_true = list(DATA_SET.loc[DATA_SET['pnum']==pnum, 'label'])
        print(np.mean(y_true))
        if np.mean(y_true) > 0.5:
            y_pred = [1] * len(y_true)
        else:    
            y_pred = [0] * len(y_true)
        
        RESULTS_AVAL.append({
                'accuracy': accuracy_score(y_true, y_pred),
                'f1_score': f1_score(y_true, y_pred)
            })
        
    return RESULTS_AVAL

def dt_model(INPUT_DATAFRAME):
    DATA_SET = INPUT_DATAFRAME

    FEATURES = DATA_SET[feature_list]
    LABELS_AVAL = DATA_SET['label']
    PIDS = DATA_SET['pnum']

    RESULTS_AVAL, MODELS_AVAL, TRAIN_SETS_AVAL = [], {}, {}

    splitter = LeaveOneGroupOut()

    for I_train, I_test in splitter.split(FEATURES, LABELS_AVAL, PIDS):    
        X_train, X_test = FEATURES.iloc[I_train], FEATURES.iloc[I_test] 
        y_train, y_test = LABELS_AVAL.iloc[I_train], LABELS_AVAL.iloc[I_test]
        test_name = np.unique(PIDS[I_test])[0]
        
        inner_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
        I_train, I_eval = next(inner_splitter.split(X_train, y_train))
        X_train, X_eval = X_train.iloc[I_train], X_train.iloc[I_eval]
        y_train, y_eval = y_train.iloc[I_train], y_train.iloc[I_eval]

        clf = DecisionTreeClassifier(criterion='entropy', min_samples_split=20)
        
        clf.fit(X_train, y_train)

        # # General approach
        # y_pred = clf.predict(X_test)
        # y_prob = clf.predict_proba(X_test)[:, 1]

        # Use of V.Mishra's two-layer approach https://dl.acm.org/doi/pdf/10.1145/3361562        
        CLF_THRESHOLD = 0.5
        GAMMA = 0.33
        DELTA = 0.86
        
        D, yi_1 = [], None
        X_test = X_test.reset_index(drop=True)
        for idx in X_test.index:
            xi = clf.predict(pd.DataFrame(X_test.iloc[idx, (X_test.columns != 'pnum') & (X_test.columns != 'cond') & (X_test.columns != 'label')]).T)[0]

            if not D or D[-1] == xi:
                yi = xi
            else:
                yi = GAMMA*(1 - yi_1)*xi + DELTA*yi_1*(1 - xi) + yi_1*xi

            Di = 1 if yi >= CLF_THRESHOLD else 0
            D.append(Di)
            yi_1 = yi

        res = filter(lambda x: x[0] != -1, zip(y_test.values, D))
        y_test, y_pred = zip(*res)

        RESULTS_AVAL.append({
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred)

        })
        MODELS_AVAL[test_name] = clf
        TRAIN_SETS_AVAL[test_name] = X_train

    return RESULTS_AVAL

def rf_model(INPUT_DATAFRAME):
    DATA_SET = INPUT_DATAFRAME

    FEATURES = DATA_SET[feature_list]
    LABELS_AVAL = DATA_SET['label']
    PIDS = DATA_SET['pnum']

    RESULTS_AVAL, MODELS_AVAL, TRAIN_SETS_AVAL = [], {}, {}

    splitter = LeaveOneGroupOut()

    for I_train, I_test in splitter.split(FEATURES, LABELS_AVAL, PIDS):    
        X_train, X_test = FEATURES.iloc[I_train], FEATURES.iloc[I_test] 
        y_train, y_test = LABELS_AVAL.iloc[I_train], LABELS_AVAL.iloc[I_test]
        test_name = np.unique(PIDS[I_test])[0]
        
        inner_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
        I_train, I_eval = next(inner_splitter.split(X_train, y_train))
        X_train, X_eval = X_train.iloc[I_train], X_train.iloc[I_eval]
        y_train, y_eval = y_train.iloc[I_train], y_train.iloc[I_eval]
        
        clf = RandomForestClassifier(criterion='gini', min_samples_split=10, n_estimators=100)
        
        clf.fit(X_train, y_train)

        # # General approach
        # y_pred = clf.predict(X_test)
        # y_prob = clf.predict_proba(X_test)[:, 1]

        # Use of V.Mishra's two-layer approach https://dl.acm.org/doi/pdf/10.1145/3361562        
        CLF_THRESHOLD = 0.5
        GAMMA = 0.33
        DELTA = 0.86
        
        D, yi_1 = [], None
        X_test = X_test.reset_index(drop=True)
        for idx in X_test.index:
            xi = clf.predict(pd.DataFrame(X_test.iloc[idx, (X_test.columns != 'pnum') & (X_test.columns != 'cond') & (X_test.columns != 'label')]).T)[0]

            if not D or D[-1] == xi:
                yi = xi
            else:
                yi = GAMMA*(1 - yi_1)*xi + DELTA*yi_1*(1 - xi) + yi_1*xi

            Di = 1 if yi >= CLF_THRESHOLD else 0
            D.append(Di)
            yi_1 = yi

        res = filter(lambda x: x[0] != -1, zip(y_test.values, D))
        y_test, y_pred = zip(*res)

        RESULTS_AVAL.append({
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred)
        })
        MODELS_AVAL[test_name] = clf
        TRAIN_SETS_AVAL[test_name] = X_train

    return RESULTS_AVAL

def ab_model(INPUT_DATAFRAME):
    DATA_SET = INPUT_DATAFRAME

    FEATURES = DATA_SET[feature_list]
    LABELS_AVAL = DATA_SET['label']
    PIDS = DATA_SET['pnum']

    RESULTS_AVAL, MODELS_AVAL, TRAIN_SETS_AVAL = [], {}, {}

    splitter = LeaveOneGroupOut()

    for I_train, I_test in splitter.split(FEATURES, LABELS_AVAL, PIDS):    
        X_train, X_test = FEATURES.iloc[I_train], FEATURES.iloc[I_test] 
        y_train, y_test = LABELS_AVAL.iloc[I_train], LABELS_AVAL.iloc[I_test]
        test_name = np.unique(PIDS[I_test])[0]
        
        inner_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
        I_train, I_eval = next(inner_splitter.split(X_train, y_train))
        X_train, X_eval = X_train.iloc[I_train], X_train.iloc[I_eval]
        y_train, y_eval = y_train.iloc[I_train], y_train.iloc[I_eval]

        dt = DecisionTreeClassifier(criterion='entropy', min_samples_split=20)
        clf = AdaBoostClassifier(estimator=dt, n_estimators=100, algorithm='SAMME.R')
        
        clf.fit(X_train, y_train)

        # # General approach
        # y_pred = clf.predict(X_test)
        # y_prob = clf.predict_proba(X_test)[:, 1]

        # Use of V.Mishra's two-layer approach https://dl.acm.org/doi/pdf/10.1145/3361562        
        CLF_THRESHOLD = 0.5
        GAMMA = 0.33
        DELTA = 0.86
        
        D, yi_1 = [], None
        X_test = X_test.reset_index(drop=True)
        for idx in X_test.index:
            xi = clf.predict(pd.DataFrame(X_test.iloc[idx, (X_test.columns != 'pnum') & (X_test.columns != 'cond') & (X_test.columns != 'label')]).T)[0]

            if not D or D[-1] == xi:
                yi = xi
            else:
                yi = GAMMA*(1 - yi_1)*xi + DELTA*yi_1*(1 - xi) + yi_1*xi

            Di = 1 if yi >= CLF_THRESHOLD else 0
            D.append(Di)
            yi_1 = yi

        res = filter(lambda x: x[0] != -1, zip(y_test.values, D))
        y_test, y_pred = zip(*res)

        RESULTS_AVAL.append({
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred)
        })
        MODELS_AVAL[test_name] = clf
        TRAIN_SETS_AVAL[test_name] = X_train

    return RESULTS_AVAL

def lda_model(INPUT_DATAFRAME):
    DATA_SET = INPUT_DATAFRAME

    FEATURES = DATA_SET[feature_list]
    LABELS_AVAL = DATA_SET['label']
    PIDS = DATA_SET['pnum']

    RESULTS_AVAL, MODELS_AVAL, TRAIN_SETS_AVAL = [], {}, {}

    splitter = LeaveOneGroupOut()

    for I_train, I_test in splitter.split(FEATURES, LABELS_AVAL, PIDS):    
        X_train, X_test = FEATURES.iloc[I_train], FEATURES.iloc[I_test] 
        y_train, y_test = LABELS_AVAL.iloc[I_train], LABELS_AVAL.iloc[I_test]
        test_name = np.unique(PIDS[I_test])[0]
        
        inner_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
        I_train, I_eval = next(inner_splitter.split(X_train, y_train))
        X_train, X_eval = X_train.iloc[I_train], X_train.iloc[I_eval]
        y_train, y_eval = y_train.iloc[I_train], y_train.iloc[I_eval]

        clf = LinearDiscriminantAnalysis()
        
        clf.fit(X_train, y_train)

        # # General approach
        # y_pred = clf.predict(X_test)
        # y_prob = clf.predict_proba(X_test)[:, 1]

        # Use of V.Mishra's two-layer approach https://dl.acm.org/doi/pdf/10.1145/3361562        
        CLF_THRESHOLD = 0.5
        GAMMA = 0.33
        DELTA = 0.86
        
        D, yi_1 = [], None
        X_test = X_test.reset_index(drop=True)
        for idx in X_test.index:
            xi = clf.predict(pd.DataFrame(X_test.iloc[idx, (X_test.columns != 'pnum') & (X_test.columns != 'cond') & (X_test.columns != 'label')]).T)[0]

            if not D or D[-1] == xi:
                yi = xi
            else:
                yi = GAMMA*(1 - yi_1)*xi + DELTA*yi_1*(1 - xi) + yi_1*xi

            Di = 1 if yi >= CLF_THRESHOLD else 0
            D.append(Di)
            yi_1 = yi

        res = filter(lambda x: x[0] != -1, zip(y_test.values, D))
        y_test, y_pred = zip(*res)

        RESULTS_AVAL.append({
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred)
        })
        MODELS_AVAL[test_name] = clf
        TRAIN_SETS_AVAL[test_name] = X_train

    return RESULTS_AVAL

def knn_model(INPUT_DATAFRAME):
    DATA_SET = INPUT_DATAFRAME

    FEATURES = DATA_SET[feature_list]
    LABELS_AVAL = DATA_SET['label']
    PIDS = DATA_SET['pnum']

    RESULTS_AVAL, MODELS_AVAL, TRAIN_SETS_AVAL = [], {}, {}

    splitter = LeaveOneGroupOut()

    for I_train, I_test in splitter.split(FEATURES, LABELS_AVAL, PIDS):    
        X_train, X_test = FEATURES.iloc[I_train], FEATURES.iloc[I_test] 
        y_train, y_test = LABELS_AVAL.iloc[I_train], LABELS_AVAL.iloc[I_test]
        test_name = np.unique(PIDS[I_test])[0]
        
        inner_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
        I_train, I_eval = next(inner_splitter.split(X_train, y_train))
        X_train, X_eval = X_train.iloc[I_train], X_train.iloc[I_eval]
        y_train, y_eval = y_train.iloc[I_train], y_train.iloc[I_eval]

        clf = KNeighborsClassifier(n_neighbors=9)
        
        clf.fit(X_train, y_train)

        # # General approach
        # y_pred = clf.predict(X_test)
        # y_prob = clf.predict_proba(X_test)[:, 1]

        # Use of V.Mishra's two-layer approach https://dl.acm.org/doi/pdf/10.1145/3361562        
        CLF_THRESHOLD = 0.5
        GAMMA = 0.33
        DELTA = 0.86
        
        D, yi_1 = [], None
        X_test = X_test.reset_index(drop=True)
        for idx in X_test.index:
            xi = clf.predict(pd.DataFrame(X_test.iloc[idx, (X_test.columns != 'pnum') & (X_test.columns != 'cond') & (X_test.columns != 'label')]).T)[0]

            if not D or D[-1] == xi:
                yi = xi
            else:
                yi = GAMMA*(1 - yi_1)*xi + DELTA*yi_1*(1 - xi) + yi_1*xi

            Di = 1 if yi >= CLF_THRESHOLD else 0
            D.append(Di)
            yi_1 = yi

        res = filter(lambda x: x[0] != -1, zip(y_test.values, D))
        y_test, y_pred = zip(*res)

        RESULTS_AVAL.append({
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred)
        })
        MODELS_AVAL[test_name] = clf
        TRAIN_SETS_AVAL[test_name] = X_train

    return RESULTS_AVAL

def svm_model(INPUT_DATAFRAME):
    DATA_SET = INPUT_DATAFRAME

    FEATURES = DATA_SET[feature_list]
    LABELS_AVAL = DATA_SET['label']
    PIDS = DATA_SET['pnum']

    RESULTS_AVAL, MODELS_AVAL, TRAIN_SETS_AVAL = [], {}, {}

    splitter = LeaveOneGroupOut()

    for I_train, I_test in splitter.split(FEATURES, LABELS_AVAL, PIDS):    
        X_train, X_test = FEATURES.iloc[I_train], FEATURES.iloc[I_test] 
        y_train, y_test = LABELS_AVAL.iloc[I_train], LABELS_AVAL.iloc[I_test]
        test_name = np.unique(PIDS[I_test])[0]

        clf = SVC(probability=True, kernel='rbf', C=107, gamma=0.001)
        
        clf.fit(X_train, y_train)

        # # General approach
        # y_pred = clf.predict(X_test)
        # y_prob = clf.predict_proba(X_test)[:, 1]

        # Use of V.Mishra's two-layer approach https://dl.acm.org/doi/pdf/10.1145/3361562        
        CLF_THRESHOLD = 0.5
        GAMMA = 0.33
        DELTA = 0.86
        
        D, yi_1 = [], None
        X_test = X_test.reset_index(drop=True)
        for idx in X_test.index:
            xi = clf.predict(pd.DataFrame(X_test.iloc[idx, (X_test.columns != 'pnum') & (X_test.columns != 'cond') & (X_test.columns != 'label')]).T)[0]

            if not D or D[-1] == xi:
                yi = xi
            else:
                yi = GAMMA*(1 - yi_1)*xi + DELTA*yi_1*(1 - xi) + yi_1*xi

            Di = 1 if yi >= CLF_THRESHOLD else 0
            D.append(Di)
            yi_1 = yi

        res = filter(lambda x: x[0] != -1, zip(y_test.values, D))
        y_test, y_pred = zip(*res)

        RESULTS_AVAL.append({
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred)
        })
        MODELS_AVAL[test_name] = clf
        TRAIN_SETS_AVAL[test_name] = X_train


        RESULTS_AVAL.append({
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred)
        })
        MODELS_AVAL[test_name] = clf
        TRAIN_SETS_AVAL[test_name] = X_train

    return RESULTS_AVAL

def xgb_model(INPUT_DATAFRAME):
    DATA_SET = INPUT_DATAFRAME

    FEATURES = DATA_SET[feature_list]
    LABELS_AVAL = DATA_SET['label']
    PIDS = DATA_SET['pnum']

    RESULTS_AVAL, MODELS_AVAL, TRAIN_SETS_AVAL = [], {}, {}

    splitter = LeaveOneGroupOut()

    for I_train, I_test in splitter.split(FEATURES, LABELS_AVAL, PIDS):    
        X_train, X_test = FEATURES.iloc[I_train], FEATURES.iloc[I_test] 
        y_train, y_test = LABELS_AVAL.iloc[I_train], LABELS_AVAL.iloc[I_test]
        test_name = np.unique(PIDS[I_test])[0]

        inner_splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
        I_train, I_eval = next(inner_splitter.split(X_train, y_train))
        X_train, X_eval = X_train.iloc[I_train], X_train.iloc[I_eval]
        y_train, y_eval = y_train.iloc[I_train], y_train.iloc[I_eval]

        clf = XGBClassifier(objective='binary:logistic', learning_rate=0.1, max_depth=3, n_estimators=100)

        clf.fit(X_train, y_train, eval_set=[(X_eval, y_eval)])

        # # General approach
        # y_pred = clf.predict(X_test)
        # y_prob = clf.predict_proba(X_test)[:, 1]

        # Use of V.Mishra's two-layer approach https://dl.acm.org/doi/pdf/10.1145/3361562        
        CLF_THRESHOLD = 0.5
        GAMMA = 0.33
        DELTA = 0.86
        
        D, yi_1 = [], None
        X_test = X_test.reset_index(drop=True)
        for idx in X_test.index:
            xi = clf.predict(pd.DataFrame(X_test.iloc[idx, (X_test.columns != 'pnum') & (X_test.columns != 'cond') & (X_test.columns != 'label')]).T)[0]

            if not D or D[-1] == xi:
                yi = xi
            else:
                yi = GAMMA*(1 - yi_1)*xi + DELTA*yi_1*(1 - xi) + yi_1*xi

            Di = 1 if yi >= CLF_THRESHOLD else 0
            D.append(Di)
            yi_1 = yi

        res = filter(lambda x: x[0] != -1, zip(y_test.values, D))
        y_test, y_pred = zip(*res)

        RESULTS_AVAL.append({
            'accuracy': accuracy_score(y_test, y_pred),
            'f1_score': f1_score(y_test, y_pred)
        })
        MODELS_AVAL[test_name] = clf
        TRAIN_SETS_AVAL[test_name] = X_train

        print(np.mean(y_pred))

    return RESULTS_AVAL

In [239]:
RESULTS_DATAFRAME_MV = major_voting(HRV_FEATURES_DATAFRAME_NORMALIZED)
print(pd.DataFrame(RESULTS_DATAFRAME_MV).mean())
print(pd.DataFrame(RESULTS_DATAFRAME_MV).std())

0.5963375796178344
0.5846438482886216
0.5872873769024172
0.6031483015741508
0.5943637916310845
0.5296803652968036
0.6060606060606061
0.6254237288135593
0.6109637488947833
0.6176991150442478
0.6121883656509696
0.6199829205807003
accuracy    0.598982
f1_score    0.748909
dtype: float64
accuracy    0.025315
f1_score    0.020359
dtype: float64


In [240]:
RESULTS_DATAFRAME_DT = dt_model(HRV_FEATURES_DATAFRAME_NORMALIZED)
print(pd.DataFrame(RESULTS_DATAFRAME_DT).mean())
print(pd.DataFrame(RESULTS_DATAFRAME_DT).std())

accuracy    0.731786
f1_score    0.779600
dtype: float64
accuracy    0.165683
f1_score    0.155150
dtype: float64


In [241]:
RESULTS_DATAFRAME_RF = rf_model(HRV_FEATURES_DATAFRAME_NORMALIZED)
print(pd.DataFrame(RESULTS_DATAFRAME_RF).mean())
print(pd.DataFrame(RESULTS_DATAFRAME_RF).std())

accuracy    0.759054
f1_score    0.795009
dtype: float64
accuracy    0.217388
f1_score    0.208652
dtype: float64


In [242]:
RESULTS_DATAFRAME_AB = ab_model(HRV_FEATURES_DATAFRAME_NORMALIZED)
print(pd.DataFrame(RESULTS_DATAFRAME_AB).mean())
print(pd.DataFrame(RESULTS_DATAFRAME_AB).std())

accuracy    0.765375
f1_score    0.808883
dtype: float64
accuracy    0.200081
f1_score    0.186254
dtype: float64


In [243]:
RESULTS_DATAFRAME_LDA = lda_model(HRV_FEATURES_DATAFRAME_NORMALIZED)
print(pd.DataFrame(RESULTS_DATAFRAME_LDA).mean())
print(pd.DataFrame(RESULTS_DATAFRAME_LDA).std())

accuracy    0.756820
f1_score    0.796696
dtype: float64
accuracy    0.250113
f1_score    0.222733
dtype: float64


In [251]:
RESULTS_DATAFRAME_kNN = knn_model(HRV_FEATURES_DATAFRAME_NORMALIZED)
print(pd.DataFrame(RESULTS_DATAFRAME_kNN).mean())
print(pd.DataFrame(RESULTS_DATAFRAME_kNN).std())

accuracy    0.652507
f1_score    0.708056
dtype: float64
accuracy    0.164101
f1_score    0.166756
dtype: float64


In [252]:
RESULTS_DATAFRAME_XGBOOST = svm_model(HRV_FEATURES_DATAFRAME_NORMALIZED)
print(pd.DataFrame(RESULTS_DATAFRAME_XGBOOST).mean())
print(pd.DataFrame(RESULTS_DATAFRAME_XGBOOST).std())

accuracy    0.747666
f1_score    0.784750
dtype: float64
accuracy    0.217345
f1_score    0.205936
dtype: float64


In [245]:
RESULTS_DATAFRAME_XGBOOST = xgb_model(HRV_FEATURES_DATAFRAME_NORMALIZED)
print(pd.DataFrame(RESULTS_DATAFRAME_XGBOOST).mean())
print(pd.DataFrame(RESULTS_DATAFRAME_XGBOOST).std())

[0]	validation_0-logloss:0.63905
[1]	validation_0-logloss:0.59401
[2]	validation_0-logloss:0.55683
[3]	validation_0-logloss:0.52586
[4]	validation_0-logloss:0.49967
[5]	validation_0-logloss:0.47896
[6]	validation_0-logloss:0.46018
[7]	validation_0-logloss:0.44360
[8]	validation_0-logloss:0.42662
[9]	validation_0-logloss:0.41241
[10]	validation_0-logloss:0.40015
[11]	validation_0-logloss:0.38639
[12]	validation_0-logloss:0.37578
[13]	validation_0-logloss:0.36686
[14]	validation_0-logloss:0.35879
[15]	validation_0-logloss:0.35020
[16]	validation_0-logloss:0.34161
[17]	validation_0-logloss:0.33592
[18]	validation_0-logloss:0.32667
[19]	validation_0-logloss:0.32006
[20]	validation_0-logloss:0.31343
[21]	validation_0-logloss:0.30979
[22]	validation_0-logloss:0.30590
[23]	validation_0-logloss:0.30185
[24]	validation_0-logloss:0.29756
[25]	validation_0-logloss:0.29397
[26]	validation_0-logloss:0.28701
[27]	validation_0-logloss:0.28482
[28]	validation_0-logloss:0.28231
[29]	validation_0-loglos

In [ ]:
# clf = XGBClassifier(objective='binary:logistic', learning_rate=0.01, max_depth=3, n_estimators=100)

# accuracy    0.760221
# f1_score    0.798257